In [1]:
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras import *
import numpy as np
from functools import partial
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from _global.config import *
from _global.funcs import *
from object_detection.utils import dataset_util

In [2]:
def _bytes_feature(value):
    # bytelist isnt able to unpack a string from an eagertensor
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [3]:
def prepare_bbxs(bbxs, width, height):
    bbx_to_transform = []
    for bbx in bbxs:
        temp = bbx
        temp[2] += temp[0]
        temp[2] /= width
        
        temp[3] += temp[1]
        temp[3] /= height

        temp[0] /= width
        temp[1] /= height

        if temp[0] > 1 or temp[1] > 1 or temp[2] > 1 or temp[2] <= temp[0] or temp[3] > 1 or temp[3] <= temp[1]:
            continue

        temp.append('x')

        bbx_to_transform.append(temp)
    return bbx_to_transform


def decode_img(img, bbxs):
    image = tf.image.decode_jpeg(img , channels=3)

    height, width, channels = image.shape
    image = tf.cast(image, tf.float32)

    image = tf.image.resize(image, [IMAGE_SIZES[0], IMAGE_SIZES[1]])
    image = image.numpy()
    if(len(bbxs) > 0):
        bbx_to_transform = prepare_bbxs(bbxs, width, height)
        transformed = transform_resize_w_bbxs(image=image, bboxes=bbx_to_transform)
        
        bbxs_with_label = transformed['bboxes']
        bbxs_return = [bbx[:4] for bbx in bbxs_with_label]

        image = tf.convert_to_tensor(transformed['image'])
    else:
        return image, [[]], [[]]
    return image, bbxs_return, bbxs_with_label

MASTER_TRANSFORM = A.Compose([
           A.OneOf([
                A.HorizontalFlip(p=1),
                A.RandomRotate90(p=1),
                A.VerticalFlip(p=1),
            ], p=0.3),
            A.OneOf([
                A.MotionBlur(10, p=1),
                A.GaussNoise((1, 60), -50 ,p=1),
                A.RandomFog(p=1),
                A.ImageCompression(1, 10, p=1.0)  
            ], p=0.4),
            A.OneOf([
                A.RandomBrightness(p=1),
                A.RandomGamma(p=1),
                A.RandomBrightnessContrast(p=1)
            ], p=0.5),
            A.RandomScale(scale_limit=0.5, p=0.3),
            A.PadIfNeeded(IMAGE_SIZES[0], IMAGE_SIZES[1], border_mode=0, value=[0,0,0]),
            A.Crop(0, 0, IMAGE_SIZES[0], IMAGE_SIZES[1], p=1)
    ], bbox_params=bbx_params)  


def random_augument(img, bbxs):
    img_temp = tf.cast(img, dtype=tf.uint8).numpy()
    transformed = MASTER_TRANSFORM(image=img_temp, bboxes=bbxs)
    return tf.convert_to_tensor(transformed['image']), transformed['bboxes']

c:\Users\niklas\Desktop\Face_Recognition_Ullsperger\LogicAIFaceRecProject\tf\lib\site-packages\albumentations\augmentations\transforms.py:1800: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [4]:
def bbx_valid(bbx):
    return (bbx[0] <= 1 and bbx[0] >=0 and bbx[1] <= 1 and bbx[1] >=0 and bbx[2] <= 1 and bbx[2] >=0
           and bbx[3] <= 1 and bbx[3] >=0 and bbx[2] > bbx[0] and bbx[3] > bbx[1])

In [5]:
def create_future(filename, bbxs, img_tensor):
    x_min = []
    x_max = []
    y_min = []
    y_max = []
    labels = []
    label_texts = []
    img_bytes = tf.io.encode_jpeg(tf.cast(img_tensor, tf.uint8))

    for bbx in bbxs:
        if not bbx_valid(bbx):
            continue
        x_min.append(bbx[0])
        y_min.append(bbx[1])
        x_max.append(bbx[2])
        y_max.append(bbx[3])
        labels.append(1)
        label_texts.append(bytes('face', 'utf-8') )

    return tf.train.Example(
        features = tf.train.Features(feature = {
            'image/filename': tf.train.Feature(bytes_list = tf.train.BytesList(value = [bytes(filename, 'utf-8') ])),
            'image/encoded': _bytes_feature(img_bytes),
            'image/height':tf.train.Feature(int64_list = tf.train.Int64List(value = [IMAGE_SIZES[1]])),
            'image/width':tf.train.Feature(int64_list = tf.train.Int64List(value = [IMAGE_SIZES[0]])),
            'image/channels':tf.train.Feature(int64_list = tf.train.Int64List(value = [3])),
            'image/format': tf.train.Feature(bytes_list = tf.train.BytesList(value = [bytes('jpeg', 'utf-8') ])),
            'image/object/bbox/xmin': tf.train.Feature(float_list = tf.train.FloatList(value = x_min)),
            'image/object/bbox/ymin': tf.train.Feature(float_list = tf.train.FloatList(value = y_min)),
            'image/object/bbox/xmax': tf.train.Feature(float_list = tf.train.FloatList(value = x_max)),
            'image/object/bbox/ymax': tf.train.Feature(float_list = tf.train.FloatList(value = y_max)),
            'image/object/class/text': tf.train.Feature(bytes_list = tf.train.BytesList(value = label_texts)),
            'image/object/class/label': tf.train.Feature(int64_list = tf.train.Int64List(value = labels)),
        })
    )

In [6]:
def augument_data(imgdata_path, curr_filename, curr_bounding_boxes):
        img_path =os.path.join(imgdata_path, curr_filename)

        bbxArr = []
        for i in curr_bounding_boxes:
            bbxArr.append([*i[:4]])

        with tf.io.gfile.GFile(img_path, 'rb') as fid:
            img_bytes = fid.read()

            img_tensor, new_bbxs, bbxs_with_label = decode_img(img_bytes, bbxs=bbxArr)
            feature_output = [create_future(curr_filename, new_bbxs, img_tensor)]

            for _ in range(4):
                tmp_img_tensor, tmp_bbxs = random_augument(img_tensor, bbxs_with_label)
                feature_output.append(create_future(curr_filename, tmp_bbxs, tmp_img_tensor))

            return feature_output

In [7]:
def convertToDataSet(labels_path, imgdata_path, output_path, limit = -1):
    with open(labels_path) as f:
        lines = f.readlines()
        curr_bounding_boxes = []
        curr_filename = ""
        first = True
        counter = 1
        prevWasBegin = False
        with tf.io.TFRecordWriter(output_path) as writer:
            for line in lines:
                if limit != -1 and counter >= limit: break

                line = line.replace("\n", "")
                if prevWasBegin:
                    prevWasBegin = False
                    continue
                
                if "/" in line:
                    if not first:
                        curr_augumented_features =  augument_data(imgdata_path, curr_filename, curr_bounding_boxes)
                        for feature in curr_augumented_features:
                            writer.write(tf.train.Example.SerializeToString(feature))
                            
                        curr_bounding_boxes = []
                        counter +=1
                    curr_filename = line
                    prevWasBegin = True
                    continue
                
                first = False

                line = line.strip()
                split = line.split(" ")
                next = []

                for num in split:
                    next.append(int(num))

                curr_bounding_boxes.append(next)
        print(counter)

In [8]:
#create validation dataset
convertToDataSet(VALIDATION_LABELS, VALIDATION_IMAGES, OUTPUT_VALIDATION_TFRECORD)

3226


In [8]:
#create train dataset
convertToDataSet(TRAIN_LABELS, TRAIN_IMAGES, OUTPUT_TRAIN_TFRECORD)

12880
